# Lab 6 - Linear models and Computational efficiency revisited
- **Author:** Guanghua Chi ([guanghua@berkeley.edu](mailto:guanghua@berkeley.edu))
- **Update:** 24 Feb 2020
- **Course:** INFO 251: Applied machine learning

### Topics:
1. Lasso vs. Ridge
2. Cross-validation to find optimal regularization parameter
3. Computational efficiency revisited

### References: 
 * [Lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso)
 * [Ridge](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge)
 * [RidgeCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV)
 * [cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score)
 * [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
 * [index series and array](http://nbviewer.jupyter.org/github/jiffyclub/blog-posts/blob/master/notebooks/indexing-series-and-arrays/indexing_series_and_arrays.ipynb)

In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

For this lab, let's keep using the [Boston Housing Prices Data Set](http://www.kellogg.northwestern.edu/faculty/weber/emp/_session_3/boston.htm).

In [2]:
from sklearn.datasets import load_boston
bdata = load_boston()

In [3]:
boston = pd.DataFrame(bdata.data)
boston.columns = bdata.feature_names[:]
boston['MEDV'] = bdata.target

In [4]:
boston.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7


In [5]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [6]:
set(boston.columns.tolist()) - set(['MEDV'])  #Set function

{'AGE',
 'B',
 'CHAS',
 'CRIM',
 'DIS',
 'INDUS',
 'LSTAT',
 'NOX',
 'PTRATIO',
 'RAD',
 'RM',
 'TAX',
 'ZN'}

In [7]:
v = list(set(boston.columns.tolist()) - set(['MEDV'])) #Like A intersection B complement
v_formular = '+'.join(v) 
v_formular #Format required for input features in R language

'INDUS+TAX+B+ZN+AGE+DIS+LSTAT+CRIM+RAD+RM+NOX+PTRATIO+CHAS'

In [8]:
model_1 = smf.ols(formula = 'MEDV ~ '+v_formular, data = boston).fit()
print(model_1.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Wed, 26 Feb 2020   Prob (F-statistic):          6.72e-135
Time:                        11:24:23   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4595      5.103      7.144      0.0

In [9]:
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True, normalize=False)
reg.fit(boston[v], boston['MEDV'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [10]:
print(reg.coef_)
print(reg.intercept_)

[ 2.05586264e-02 -1.23345939e-02  9.31168327e-03  4.64204584e-02
  6.92224640e-04 -1.47556685e+00 -5.24758378e-01 -1.08011358e-01
  3.06049479e-01  3.80986521e+00 -1.77666112e+01 -9.52747232e-01
  2.68673382e+00]
36.45948838508984


In [11]:
reg.score(boston[v], boston['MEDV'])

0.7406426641094094

In [12]:
pd.DataFrame({'variable':v, 'coefficient':reg.coef_})

,variable,coefficient
0,INDUS,0.020559
1,TAX,-0.012335
2,B,0.009312
3,ZN,0.046420
4,AGE,0.000692
5,DIS,-1.475567
6,LSTAT,-0.524758
7,CRIM,-0.108011
8,RAD,0.306049
9,RM,3.809865


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(boston[v], boston['MEDV'], test_size=0.34, random_state=4973)
reg = linear_model.LinearRegression(fit_intercept=True, normalize=False)
reg.fit(X_train, y_train)
print("R square of testing set: {:.2f}".format(reg.score(X_test, y_test)))

R square of testing set: 0.67


# LASSO

Recall that the cost function is:

$ J(\theta) =  \frac{1}{2n}||Y - X\theta||^2_2 + \lambda ||\theta||_1$

In [30]:
clf = linear_model.Lasso(alpha=0.5)
clf.fit(X_train, y_train)
print("R square of testing set: {:.2f}".format(clf.score(X_test, y_test)))
# predict
y_test_predict = clf.predict(X_test)

R square of testing set: 0.63


In [31]:
print(clf.coef_)

[-0.         -0.01718414  0.00612987  0.06204578  0.01985201 -0.83519979
 -0.62699718 -0.09099612  0.23916705  1.99743532 -0.         -0.81521692
  0.        ]


# Ridge

Recall that the cost function is:

$ J(\theta) =  \frac{1}{2n}||Y - X\theta||^2_2 + \lambda \theta ^ 2$

In [32]:
reg = linear_model.Ridge(alpha = 0) #alpha is lambda
reg.fit(X_train, y_train)
print("R square of testing set: {:.2f}".format(reg.score(X_test, y_test)))

R square of testing set: 0.67


In [33]:
reg = linear_model.Ridge(alpha = 0.5)
reg.fit(X_train, y_train)
print("R square of testing set: {:.2f}".format(reg.score(X_test, y_test)))
print(reg.coef_)

R square of testing set: 0.66
[ 1.12920313e-02 -1.47251850e-02  5.98703640e-03  5.75113345e-02
  9.22630240e-03 -1.24544882e+00 -5.09882962e-01 -1.05576156e-01
  2.50242108e-01  3.25047675e+00 -1.11410908e+01 -9.32357241e-01
  2.98300197e+00]


# Cross validation

In [34]:
(np.arange(0,10,0.1))

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
       2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8,
       3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1,
       5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4,
       6.5, 6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7,
       7.8, 7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. ,
       9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])

In [35]:
#Link: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html
reg = linear_model.RidgeCV(alphas=np.arange(0, 10, 0.1), scoring='r2', cv=10) #cv parameter to specify number of folds
reg.fit(X_train, y_train)
print("Best alpha: {}".format(reg.alpha_))
print("R square of testing set: {:.2f}".format(reg.score(X_test, y_test)))

Best alpha: 0.0
R square of testing set: 0.67


In [36]:
from sklearn.model_selection import cross_val_score
reg = linear_model.Ridge(alpha = 0)
scores = cross_val_score(reg, X_train, y_train, cv=10, scoring='r2')
print(scores)
print('mean r square: {:.2f}'.format(np.mean(scores)))
# for loop to test different alpha
# select the one with largest r square or other score

[0.75552314 0.55432303 0.79088912 0.8326933  0.70129104 0.21498305
 0.80458809 0.79212794 0.74954723 0.77798836]
mean r square: 0.70


In [37]:
#Link: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
from sklearn.model_selection import GridSearchCV
alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
model = linear_model.Ridge()
# param_grid: 
# Dictionary with parameters names (string) as keys and lists of parameter settings to try as values
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas), cv=10, scoring='r2') #Same as ridge cv but very useful when we need to tune multiple hyperparameters
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.best_params_)

0.6974211034259469
{'alpha': 0.0}


# Computational efficiency

## vectorized computation (broadcasting)

* Basic operations on numpy arrays (addition, etc.) are elementwise. This works on arrays of the same size.
* Nevertheless, It’s also possible to do operations on arrays of different sizes if Numpy can transform these arrays so that they all have the same size: this conversion is called **broadcasting**.

### [General Broadcasting Rules](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html)

When operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing dimensions, and works its way forward. Two dimensions are compatible when
* they are equal, or
* one of them is 1

![title](http://www.scipy-lectures.org/_images/numpy_broadcasting.png)

# Rules of Broadcasting
Broadcasting in NumPy follows a strict set of rules to determine the interaction between the two arrays:

### Rule 1: 
If the two arrays differ in their number of dimensions, the shape of the one with fewer dimensions is padded with ones on its leading (left) side.
### Rule 2: 
If the shape of the two arrays does not match in any dimension, the array with shape equal to 1 in that dimension is stretched to match the other shape.
### Rule 3: 
If in any dimension the sizes disagree and neither is equal to 1, an error is raised.

Image source: http://www.scipy-lectures.org/intro/numpy/numpy.html

In [50]:
a = np.array([[ 0,  0,  0],
              [10, 10, 10],
              [20, 20, 20],
              [30, 30, 30]])advant
print(a)
print(a.shape)

[[ 0  0  0]
 [10 10 10]
 [20 20 20]
 [30 30 30]]
(4, 3)


In [51]:
b = np.array([[0, 1, 2],
              [0, 1, 2],
              [0, 1, 2],
              [0, 1, 2]])
print(b)
print(b.shape)

[[0 1 2]
 [0 1 2]
 [0 1 2]
 [0 1 2]]
(4, 3)


In [52]:
a + b

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

In [53]:
b_2 = np.array([0, 1, 2])
print(b_2)
print(b_2.shape)

[0 1 2]
(3,)


In [54]:
a + b_2 #b_2 will be viewed as shape (1,3)

[[ 0  0  0]
 [10 10 10]
 [20 20 20]
 [30 30 30]]


array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

In [55]:
a_2 = np.array([[0],
                [10],
                [20],
                [30]])
print(a_2)
print(a_2.shape)

[[ 0]
 [10]
 [20]
 [30]]
(4, 1)


In [56]:
a_2 + b_2

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

In [57]:
a_2 + b

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

If these conditions are not met, a ValueError: frames are not aligned exception is thrown, indicating that the arrays have incompatible shapes.

In [58]:
print(b_2)
print(b_2.shape)

[0 1 2]
(3,)


In [59]:
b_3 = np.array([3, 4, 5, 6])
print(b_3)
print(b_3.shape)

[3 4 5 6]
(4,)


In [45]:
# the above two conditions are not met.
b_2 + b_3

ValueError: operands could not be broadcast together with shapes (3,) (4,) 

## 1. calculate the distance

In [75]:
#```python
# vectorized computation
def distance(x1, x2, L): #Specify L-norm number
    x1 = np.asarray(x1)
    x2 = np.asarray(x2)
    dist = np.power(np.sum(np.power(np.abs(x1 - x2), L), axis=1), 1.0/L)
    return dist
#```

In [77]:
a= np.array([[1,2,3,4], [10,20,30,40]]) #2 examples in your dataset
b=np.array([5,6,7,8]) #A vector from which distance needs to be calculated for all examples
distance(a,b,2)

array([ 8.        , 42.11887938])

## 2. calculate RMSE

In [39]:
#```python
def compute_rmse(predictions, yvalues):
    pre = np.asarray(predictions)
    y = np.asarray(yvalues)
    rmse = np.sqrt(np.sum((pre-y) ** 2) / float(len(y)))
    return rmse
#```

In [41]:
y_hat = [1,2,4,5]
y=[1,2,3,4]
compute_rmse(y_hat, y)

0.7071067811865476

## 3. normalize

In [44]:
#```python
def standardize(raw_data):
    return ((raw_data - np.mean(raw_data, axis = 0)) / np.std(raw_data, axis = 0))
#```

In [45]:
a= np.array([[1,2,3,4], [10,20,30,40], [20, 21, 22, 23]])
a

array([[ 1,  2,  3,  4],
       [10, 20, 30, 40],
       [20, 21, 22, 23]])

In [46]:
standardize(a)

array([[-1.20270298, -1.41266656, -1.35411306, -1.24678415],
       [-0.04295368,  0.64906302,  1.03030342,  1.20144654],
       [ 1.24565666,  0.76360355,  0.32380965,  0.04533761]])